# Customer Lifetime Valuation

This is a python notebook constructed by Lennert Van der Schraelen, Bjarne Brié, Prof. Dr. Kristof Stouthuysen and Prof. Dr. Tim Verdonck. All authors are members of the **Centre for Financial Leaderschip and Digital Transformation** at **Vlerick Business School**.

Contact information: 
- lennert.vanderschraelen@vlerick.com
- bjarne.brie@vlerick.com
- kristof.stouthuysen@vlerick.com
- tim.verdonck@uantwerpen.be 

## Introduction

<center>
<img src='https://www.dropbox.com/s/uq49tl5f8c6z39i/graph.PNG?raw=1' width='850' class='center'>
<center>

### Managerial Relevance

**Identify Customer Lifetime Value (CLV) of different customers or customer segments.**

Devise strategies that:
 * Add value to most profitable AND high potential customers;
 * Make lower CLV customers more valuable by enhancing and accelerating cash streams;
 * Stop or reduce the erosion of value by loss-making customers.

### How?

**A customer is considered to be an asset to the company.**

* Valuation of that asset:

<script src='https://cdnjs.cloudflare.com/ajax/libs/mathjax/2.7.4/MathJax.js?config=default'></script> $$ \text{CLV}=\sum_{t=1}^{\text{T}}\dfrac{p_t - c_t}{(1+i)^t} - \text{AC}. $$

* <script src='https://cdnjs.cloudflare.com/ajax/libs/mathjax/2.7.4/MathJax.js?config=default'></script> $p_t$ = price paid by a customer at time t
* <script src='https://cdnjs.cloudflare.com/ajax/libs/mathjax/2.7.4/MathJax.js?config=default'></script> $c_t$ = discount rate or cost of capital for the firm
* <script src='https://cdnjs.cloudflare.com/ajax/libs/mathjax/2.7.4/MathJax.js?config=default'></script> $\frac{1}{(1+i)^t}  $ = time horizon for estimating CLV
* AC = acquisition cost
* <script src='https://cdnjs.cloudflare.com/ajax/libs/mathjax/2.7.4/MathJax.js?config=default'></script> $T$ = time horizon for estimating CLV


<center>
<img src='https://www.dropbox.com/s/lwl6m9psa7e5y65/hair.png?raw=1' width='250' class='center'>
<center>

## Case Eurotel Part B - Customer Lifetime Valuation

### Case questions

* How can the churn probabilities be used to determine the expected customer relationship duration? 
* Examine and explain the CLV calculation for an individual Eurotel customer.
* Targeting customer and product segments
    - Determine the potential acquisition value for each product and customer segment. Which segments are most attractive?
    - Determine the five product and customer segments that Eurotel should target?
* Which marketing campaigns should Eurotel invest in?

### Import packages and read in the dataset

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
df_eurotel = pd.read_csv('../data/Eurotel_churn_output_data.csv')
df_eurotel.head(5)

### Data description

The <a href='https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.describe.html'>`df.describe()`</a> method can be used to calculate basic descriptive statistics.

### Data cleaning

To ease interpretation, we replace `InternetService_No` with `InternetService`. 

In [ ]:
df_eurotel['InternetService_No'] = \
    df_eurotel['InternetService_No'].replace({0: 1, 1: 0})
df = df_eurotel.rename(columns={'InternetService_No':
                        'InternetService'})
df.head(5)

## CLV analysis

Questions:

* How can the churn probabilities be used to determine the expected customer relationship duration?
* How can we calculate the CLV values per customer?

Some important information that can be found in the case:

<ul>
  <li>The company uses a yearly WACC of 8%.</li>
  <li>The CLV analysis is EBITDA-based.</li>
</ul>

### Customer lifetime

Important in any CLV analysis is the time horizon under consideration. For each customer, we can determine the expected **customer relationship duration (CRD)** based on that customer's probability of churning:

For each customer j:

<script src='https://cdnjs.cloudflare.com/ajax/libs/mathjax/2.7.4/MathJax.js?config=default'></script> $$ CRD_j=\dfrac{1}{P(churn)}. $$
<br>
</br>
Let's calculate the CRD and include this information in a new column `CRD`. We work with rounded values.

### Customer lifetime

Some customers have a very high CRD (and thus a very low churn probability). This means that they are very likely to stick with our company for a long time. 

Others have a very high churn probability and thus a low CRD. These people have a high probability of switching to a competitor. 

Let's take a look at the minimum and maximum CRD in our data.

In [ ]:
min_crd = int(min(df['CRD']))
max_crd = int(max(df['CRD']))

print('The minimum CRD is:', min_crd)
print('The maximum CRD is:', max_crd)

### CLV calculation

To compute the CLV, we need information on Eurotel's WACC as well as the forecasted EBITDA rates. Let's import the Excel file that contains all the exhibits and extract the sheets that contain the EBITDA as a percentage of monthly revenue and the WACC. 

We use the <a href='https://pandas.pydata.org/docs/reference/api/pandas.ExcelFile.parse.html'>pd.ExcelFile.parse()</a> function that parses specified sheet(s) into a pandas DataFrame.

In [ ]:
exhibits = pd.ExcelFile( '../data/Eurotel_exhibit_data.xlsx')
month_wacc = exhibits.parse('WACC_month')
ebitda_perc = exhibits.parse('Exhibit 2',index_col=0)
print(month_wacc)
print(ebitda_perc)

### CLV calculation

We can now calculate the CLV values per customer. 

To do so, we use the <a href='https://numpy.org/numpy-financial/'>numpy_financial</a> package.This package contains the <a href='https://numpy.org/numpy-financial/latest/npv.html'>npf.npv()</a> function that returns the NPV (Net Present Value) of a cash flow stream.

In [ ]:
import numpy_financial as npf

<script src='https://cdnjs.cloudflare.com/ajax/libs/mathjax/2.7.4/MathJax.js?config=default'></script> $$ \text{CLV}=\sum_{i=1}^{\text{CRD}}\dfrac{\text{EBITDA}_{{\text{month}}_i}}{(1+\text{WACC}_{\text{month}})^i}. $$




### CLV calculation

First, we construct two empty arrays that we use to store our calculations. Are you able to explain the shape of these arrays? 

Next, we extract the WACC from the pandas Series and store the EBITDA data in the predefined array. 

In [ ]:
array_ebitda_perc = np.zeros(max_crd) 

amount_clients = len(df)
clv = np.zeros(amount_clients) 

month_wacc = np.array(month_wacc['WACC_month'])
      
array_ebitda_perc[0] = ebitda_perc['M1']
array_ebitda_perc[1] = ebitda_perc['M2']
array_ebitda_perc[2] = ebitda_perc['M3']
array_ebitda_perc[3] = ebitda_perc['M4']
array_ebitda_perc[4] = ebitda_perc['M5']
array_ebitda_perc[5:max_crd] = ebitda_perc['M6-inf']

### CLV calculation

Second, we construct arrays that contain the revenues per user and the customer relationship durations. 

We focus on the first three customers to gather some insight. Afterwards we do the same calculation for all customers.

Let's take a look at the monthly revenue of the first 3 customers and their CRD. 

In [ ]:
revenueperuser = np.array(df['RevenuePerUser'])
crd = np.array(df['CRD']).astype(int)

clv3 = np.zeros(3) 
revenueperuser3 = revenueperuser[0:3]
crd3 = crd[0:3]
print(revenueperuser3)
print(crd3)

### CLV calculation

Third, we multiply the EBITDA percentage (different for each month) with the monthly revenue.

To do so, we use the <a href='https://numpy.org/doc/stable/reference/generated/numpy.einsum.html'>np.einsum()</a> function. This function uses the Einstein summation convention, that enables us to represent array operations in a simple fashion.

We store this information in the two-dimensional array `ebitda3`. Can you  explain the shape of this array?

In [ ]:
ebitda3 = np.einsum('i,j->ji', 
    array_ebitda_perc, revenueperuser3)
print(ebitda3)
print(ebitda3.shape)

### CLV calculation

We now have all the information to compute the CLV. To do so, we discount the EBITDA values using the monthly WACC and sum over the lenght of the CRD.

For instance for the second customer (CRD = 2):
    
<script src='https://cdnjs.cloudflare.com/ajax/libs/mathjax/2.7.4/MathJax.js?config=default'></script> \begin{align} \text{CLV}&=\dfrac{30.616}{1+\text{WACC}_{\text{month}}} \\ &+ \dfrac{28.48}{(1+\text{WACC}_{\text{month}})^2}. \end{align}.


In [ ]:
for i, val in enumerate(crd3):

    '''
    Complete this piece of code
    '''
    
clv3

### CLV calculation

We now do the same calculations for all customers and include the CLV values in our DataFrame. 

In [ ]:
matrix = np.einsum('i,j->ji', array_ebitda_perc, revenueperuser)

for i, val in enumerate(crd):

    clv[i] = npf.npv(month_wacc, matrix[i,0:val])

df['CLV']=clv
df.head(3)

### Customer and product segments

What is the distribution of customers in each product segment?

Customer segments:

<ul>
  <li>Youth</li>
  <li>Adults without children</li>
  <li>Adults with children</li>
  <li>Seniors</li>
</ul>

Product segments:

<ul>
  <li>Mobile</li>
  <li>Web</li>
  <li>Mobile & Web</li>
  <li>Web & TV</li>
  <li>Web & Streaming</li>
  <li>Web & Streaming & TV</li>
  <li>Mobile & Web & Streaming</li>
  <li>Mobile & Web & TV</li>
  <li>MaxPack</li>
</ul>


### Customer and product segments

In order to capture segment information, the data need to be transformed.

In [ ]:
for i, row in df.iterrows():
    if df.loc[i,'SeniorCitizen'] == 1:
        df.loc[i,'CustomerSegment'] = 'Senior'
    elif df.loc[i,'Young'] == 1:
        df.loc[i,'CustomerSegment'] = 'Youth'
    elif df.loc[i,'Dependents'] == 1:
        df.loc[i,'CustomerSegment'] = 'Adults-Children'
    else :
        df.loc[i,'CustomerSegment'] = 'Adults-NoChildren'
        
for i, row in df.iterrows():
    if df.loc[i,'InternetService'] == 0 and df.loc[i,'PhoneService'] == 1: 
        df.loc[i,'ProductSegment'] = 'Mobile'
    elif df.loc[i,'InternetService'] == 1 and df.loc[i,'PhoneService'] == 0  \
        and df.loc[i,'StreamingTV']== 0 and df.loc[i,'StreamingMovies']== 0:
        df.loc[i,'ProductSegment'] = 'Web'
    elif df.loc[i,'InternetService'] == 1 and df.loc[i,'PhoneService'] == 1 \
        and df.loc[i,'StreamingTV']== 0 and df.loc[i,'StreamingMovies']== 0:
        df.loc[i,'ProductSegment'] = 'Mobile+Web'
    elif df.loc[i,'InternetService'] == 1 and df.loc[i,'PhoneService'] == 0 \
        and df.loc[i,'StreamingTV']== 1 and df.loc[i,'StreamingMovies']== 0:
        df.loc[i,'ProductSegment'] = 'Web+TV'
    elif df.loc[i,'InternetService'] == 1 and df.loc[i,'PhoneService'] == 0 \
        and df.loc[i,'StreamingTV']== 0 and df.loc[i,'StreamingMovies']== 1:
        df.loc[i,'ProductSegment'] = 'Web+Streaming'
    elif df.loc[i,'InternetService'] == 1 and df.loc[i,'PhoneService'] == 0 \
        and df.loc[i,'StreamingTV']== 1 and df.loc[i,'StreamingMovies']== 1:
        df.loc[i,'ProductSegment'] = 'Web+Streaming+TV'
    elif df.loc[i,'InternetService'] == 1 and df.loc[i,'PhoneService'] == 1 \
        and df.loc[i,'StreamingTV']== 0 and df.loc[i,'StreamingMovies']== 1:
        df.loc[i,'ProductSegment'] = 'Mobile+Web+Streaming'
    elif df.loc[i,'InternetService'] == 1 and df.loc[i,'PhoneService'] == 1 \
        and df.loc[i,'StreamingTV']== 1 and df.loc[i,'StreamingMovies']== 0:
        df.loc[i,'ProductSegment'] = 'Mobile+Web+TV'
    else :
        df.loc[i,'ProductSegment'] = 'MaxPack'

### Customer and product segments

A crosstable (see <a href='https://pandas.pydata.org/docs/reference/api/pandas.crosstab.html'>df.crosstab()</a>) creates by default a frequency table of the factors you provide. Let's construct a crosstable that displays the number of customers in the different segments. Do you observe any interesting patterns?

## Exercise 

In the next part of the notebook, some of the blocks of code are intentionally left blank. Use the snippets of code at the bottom of the notebook to complete the notebook. The description above each empty cell hints at what piece of code should be put in that cell. Sometimes you will have to write some code yourself to complete a piece of code (this is always indicated above the corresponding piece of code with a comment). 

Good luck! 

## Case question 3: targeting customer and product segments

Goal: propose which five product/customer segments should be prioritized for marketing efforts. 

Group the customers in segments to calculate:

* The total CLV of a segment
* The number of customers in a segment
* The average CLV of a segment


In [ ]:
df_grouped_cust = df[['CustomerSegment', 'CLV']].copy()
df_grouped_cust['Count'] = ''

segment_per_customer = df_grouped_cust.groupby([ 'CustomerSegment']). CLV.sum().reset_index()
segment_per_customer['Count'] = df_grouped_cust.groupby(['CustomerSegment']).count().reset_index()['Count']
segment_per_customer['AvgCLVCustSeg'] =  segment_per_customer['CLV']/segment_per_customer['Count']
segment_per_customer

Do the same for the product segments.

We have obtained the CLV values per customer segment and per product segment. These will be used in the further analysis.

Note that the CLV values per customer segment and per product segment already reveal interesting information. Can you draw some conclusions?

To determine the potential acquisition value for each product segment, we need to calculate the number of clients that could potentially switch from the competitor.

In [ ]:
switch_prob = exhibits.parse('Exhibit 3',index_col=0)
size_prod_seg = exhibits.parse('Exhibit 1', index_col=0)
size_prod_seg = size_prod_seg.drop('TOTAL', axis=1)

switch_per_seg = pd.DataFrame(switch_prob.values
 *size_prod_seg.values, columns=switch_prob.columns,  index=switch_prob.index)
switch_per_seg['Total Potential Acquired Customers'] = switch_per_seg.sum(axis=1)
switch_per_seg

Knowing this, we can now obtain the potential acquisition value per product segment.

<center>
<img src='https://www.dropbox.com/s/ngpqkuszu5nlbqr/cust_seg_meme.jpg?raw=1' width='400' class='center'>
<center>

Let's construct a DataFrame that shows the percentages of customers in each product segment. 

In [ ]:
percen_cus = pd.crosstab(index=df['ProductSegment'], columns=df['CustomerSegment']).apply(lambda r: r/r.sum(), axis=1)
percen_cus = percen_cus.rename(columns={'Adults-Children': 'Adults-Children-Percent', 'Adults-NoChildren': 'Adults-NoChildren-Percent'
                           ,'Senior': 'Senior-Percent','Youth': 'Youth-Percent'})
percen_cus

Import the strategic importance scores.

We can now calculate the value potential of the different product/customer groups. While doing so, we also keep track of the strategic importance scores.

In [ ]:
pot_acq_val= per_segment_CLV['Potential acquired Value']
seg_pot = percen_cus.mul(pot_acq_val, axis=0).rename(columns={'Adults-Children-Percent': 'Adults-Children-Potential', 
                                                              'Adults-NoChildren-Percent': 'Adults-NoChildren-Potential'
                                       ,'Senior-Percent': 'Senior-Potential','Youth-Percent': 'Youth-Potential'})
mark_prio = pd.concat([percen_cus, seg_pot], axis = 1)
mark_prio['Strategic Importance Score'] = strat_imp_score
mark_prio

Construct a heatmap visualising the segment potentials. Based on this heatmap and the data matrix, which segments would you prioritize? 

**Hint:** you can always ask ChatGPT to *'create a heatmap from a pandas dataframe'* to get an idea what the code should look like.

## Case question 4: assessment of marketing campaigns

<center>
<img src='https://www.dropbox.com/s/ldy27rng7fxsfyg/marketing.jpg?raw=1' width='450' class='center'>
<center>

Now that we identified which 5 segments to prioritize, it's time to analyze how we will target each of those segments.  

To do so, we first load in the data on the different marketing campaigns (Exhibit 5). 

In [ ]:
mark_camp = exhibits.parse('Exhibit 5',index_col=0)
mark_camp

Next, we need to define the target acquisition rates.

Let's visualize what the dataframes *rates* and *costs* look like:

In [ ]:
rates

In [ ]:
costs

Calculate the number of target customers in each segment.

In [ ]:
pot_cust = per_segment_CLV['Total Potential Acquired Customers']
amount_cust = percen_cus.mul(pot_cust, axis=0).rename(columns={'Adults-Children-Percent': 'Adults-Children-Target', 
                                                              'Adults-NoChildren-Percent': 'Adults-NoChildren-Target'
                                       ,'Senior-Percent': 'Senior-Target','Youth-Percent': 'Youth-Target'})

amount_cust = amount_cust.round()
amount_cust

Extract the 5 product/customer segments which you selected in the previous section (question 3).

In [ ]:
# This piece of code is not complete, ' ' indicates empty strings that you should fill in 
# The values that you need to fill in correspond to row and column names from the previous output
# So, segment 1 = amount_cust.loc['row_name', 'column_name']

segment1 = amount_cust.loc[' ', ' ']
segment2 = amount_cust.loc[' ', ' ']
segment3 = amount_cust.loc[' ', ' ']
segment4 = amount_cust.loc[' ', ' ']
segment5 = amount_cust.loc[' ', ' ']

# The next step is to store these data into a data frame
# Feel free to change the names of the columns to a value of your choice
all_camp = pd.DataFrame(np.array([[segment1,segment2,segment3,segment4,segment5]]),
                        index =['Potential Acquired Customers'],
                        columns=['segment_1', 'segment_2', 'segment_3','segment_4', 'segment_5'])

all_camp

Use the target segment acquisition rates for the different campaigns to make an overview of acquired customers.

In [ ]:
# This piece of code is not complete, ' ' indicates empty strings that you should fill in 
# The values that you need to fill in correspond to column names of the 'rates' data frame
# The column names should be those that best correspond to your chosen segments 

number_pot_acq_cust_segment1 =  segment1 * rates[' '].rename('Potential-Acquired-Segment1')
number_pot_acq_cust_segment2 =  segment2 * rates[' '].rename('Potential-Acquired-Segment2')
number_pot_acq_cust_segment3 =  segment3 * rates[' '].rename('Potential-Acquired-Segment3')
number_pot_acq_cust_segment4 =  segment4 * rates[' '].rename('Potential-Acquired-Segment4')
number_pot_acq_cust_segment5 =  segment5 * rates[' '].rename('Potential-Acquired-Segment5')
pot_ac_cust = pd.concat([number_pot_acq_cust_segment1,number_pot_acq_cust_segment2,number_pot_acq_cust_segment3,
          number_pot_acq_cust_segment4,number_pot_acq_cust_segment5], axis=1)
pot_ac_cust

Finally, we can calculate the ROI for the different campaigns.

In [ ]:
# This piece of code is not complete, ' ' indicates empty strings that you should fill in. 
# The values that you need to fill in for avg_clv_prod_seg.loc[' '] correspond to the relevant row names of the avg_clv_prod_seg dataframe
# The avg_clv_prod_seg dataframe is a subset of the per_segment_CLV dataframe containing only the 'CLV' column
# These values are also similar to the product segments you selected earlier
# For the costs[' '], you need to fill in the relevant columns from the costs dataframe

avg_clv_prod_seg = per_segment_CLV['CLV']

roi_segment1 = (avg_clv_prod_seg.loc[' ']*number_pot_acq_cust_segment1)/costs[' '] - 1
roi_segment2 = (avg_clv_prod_seg.loc[' ']*number_pot_acq_cust_segment2)/costs[' '] - 1
roi_segment3 = (avg_clv_prod_seg.loc[' ']*number_pot_acq_cust_segment3)/costs[' '] - 1
roi_segment4 = (avg_clv_prod_seg.loc[' ']*number_pot_acq_cust_segment4)/costs[' '] - 1
roi_segment5 = (avg_clv_prod_seg.loc[' ']*number_pot_acq_cust_segment5)/costs[' '] - 1
roi_marketing_camp = pd.concat([roi_segment1,roi_segment2,roi_segment3,
          roi_segment4,roi_segment5], axis=1)

roi_marketing_camp = roi_marketing_camp.rename(columns={0: 'ROI-Segment1', 1: 'ROI-Segment2',
                    2: 'ROI-Segment3', 3: 'ROI-Segment4', 4: 'ROI-Segment5'})

roi_marketing_camp

Construct a heatmap to visualise the different ROI values.

Based on this heatmap, which marketing campaings will you adopt? What are your conclusions? 

### Pieces of code

Below we provide the pieces of code that should be used to complete the notebook. Note, however, that the pieces of code are ordered randomly. 

In [ ]:
strat_imp_score = exhibits.parse('Exhibit 4',index_col=0)
strat_imp_score

In [ ]:
ax = sns.heatmap(mark_prio[['Adults-Children-Potential','Adults-NoChildren-Potential','Senior-Potential', 'Youth-Potential']], linewidth=0.5,cmap='YlGnBu')
ax.set_facecolor('red')
ax.set_title('Heatmap Customer Segments')
plt.show()

In [ ]:
total_pot_acq_cust = pd.DataFrame(switch_per_seg[
    'Total Potential Acquired Customers'])
per_segment_CLV = df.groupby(['ProductSegment']).CLV.mean().reset_index().set_index('ProductSegment')
per_segment_CLV = pd.concat([per_segment_CLV,total_pot_acq_cust], axis = 1)
per_segment_CLV['Potential acquired Value'] = per_segment_CLV.product(axis=1)
per_segment_CLV

In [ ]:
ax = sns.heatmap(roi_marketing_camp, linewidth=0.5,cmap='YlGnBu')
ax.set_facecolor('red')
ax.set_title('Heatmap Marketing Campaigns')
plt.show()

In [ ]:
rates = mark_camp[['Youth-Rate','Adults-NoChildren-Rate','Adults-Children-Rate', 'Seniors-Rate']]
costs = mark_camp[['Youth-Cost','Adults-NoChildren-Cost','Adults-Children-Cost', 'Seniors-Cost']]
print(rates)
print(costs)

In [ ]:
df_grouped_prod = df[['ProductSegment', 'CLV']].copy()
df_grouped_prod['Count'] = ''

segment_per_product = df_grouped_prod.groupby(['ProductSegment']).CLV.sum().reset_index()
segment_per_product['Count'] = df_grouped_prod.groupby(['ProductSegment']).count().reset_index()['Count']
segment_per_product['AvgCLVProductSeg'] = segment_per_product['CLV']/segment_per_product['Count']
segment_per_product